In [11]:
# Import libraries
import numpy as np
import pandas as pd
import string
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# Define the target group for analysis


In [12]:
# Input parameters
export_feedback_table = False
inputfile_assignment = 'rawdata/prelim_assignment_20231114.csv'
inputfile_feedback   = 'rawdata/prelim_feedback_20231114.csv'
outputfile_feedback  = 'output/prelim_feedback_20231114.xlsx'

In [13]:
# Import raw data and converting 'created' to datetime
df_assignment = pd.read_csv(inputfile_assignment, index_col=0)
df_feedback   = pd.read_csv(inputfile_feedback, index_col=0)
df_assignment['created'] = pd.to_datetime(df_assignment['created'])
df_feedback['created']   = pd.to_datetime(df_feedback['created'])

In [14]:
def calculate_time_difference(row, df_a):
    sid = row['assignment_id'] 
    action_time = row['created']
    # Check if student_id exists in df_a
    if sid in df_a.index:
        starting_time = df_a.at[sid, 'created']
        return (action_time - starting_time).total_seconds()
    else:
        return None  # or some other default value

df_feedback['time_elapsed'] = df_feedback.apply(lambda row: calculate_time_difference(row, df_assignment), axis=1)

In [15]:
# Sort the DataFrame by 'assignment_id' and 'created' to ensure the order
df_feedback = df_feedback.sort_values(by=['assignment_id', 'created'])

# Add a new column 'sequence' which enumerates each student's rows starting from 1
df_feedback['count'] = df_feedback.groupby('assignment_id').cumcount() + 1

In [16]:
# Use 911e7cad-2261-4755-825d-dd734c8bfaa1 for testing
# Find the starting time of the student
print(df_assignment.loc['911e7cad-2261-4755-825d-dd734c8bfaa1','created'])
df_feedback[df_feedback['assignment_id']=='911e7cad-2261-4755-825d-dd734c8bfaa1']

2023-11-02 09:24:41.273000+02:00


,assignment_id,phase,type_id,feedback_text,details,created,time_elapsed,count
action_id,,,,,,,,
bb718593-9649-47da-b58b-585410a0beb4,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hyvä edistys! Olet valinnut 1 pääkohdan tästä ...,"""{\""text_id\"": 309, \""feedback_code\"": 1, \""de...",2023-11-02 09:32:03.971000+02:00,442.698,1
464bcbe8-7f0f-4ea6-8002-965ed2c57289,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hyvä edistys! Olet valinnut 1 pääkohdan tästä ...,"""{\""text_id\"": 308, \""feedback_code\"": 1, \""de...",2023-11-02 09:38:06.485000+02:00,805.212,2
38bfb2e2-5519-4d03-b559-9a6ecd5f4526,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hienosti yritetty! Olet valinnut 4 pääkohtaa t...,"""{\""text_id\"": 308, \""feedback_code\"": 1, \""de...",2023-11-02 09:39:21.358000+02:00,880.085,3
074554b8-a002-43a6-ae2a-5cb0ee5863ec,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hyvä yritys! Olet valinnut 2 pääkohtaa tästä t...,"""{\""text_id\"": 308, \""feedback_code\"": 1, \""de...",2023-11-02 09:53:01.472000+02:00,1700.199,4


In [17]:
# Export to Excel
if (export_feedback_table):
    # Convert 'created' column to timezone-naive datetimes as Excel does not support timezone
    df_feedback_export = df_feedback.copy()
    df_feedback_export['created'] = df_feedback_export['created'].dt.tz_localize(None)
    df_feedback_export.to_excel(outputfile_feedback)

In [18]:
# Group by 'count' and calculate the mean of 'time_elapsed' for each group
average_time_elapsed_per_count = df_feedback.groupby('count')['time_elapsed'].describe()

# Display the average time elapsed for counts from 1 to 15
average_time_elapsed_per_count.loc[1:15]

,count,mean,std,min,25%,50%,75%,max
count,,,,,,,,
1,134.0,623.482097,472.156359,55.473,309.58800,482.6550,857.53125,2400.208
2,104.0,948.035827,497.730367,90.888,572.52275,870.7815,1214.62925,2245.239
3,74.0,1078.005405,488.426871,205.383,748.03225,1032.4765,1314.26700,2302.412
4,47.0,1118.632149,486.556060,211.424,794.88700,1111.0440,1376.47550,2248.365
5,28.0,1163.117536,379.610260,216.939,963.66125,1157.2980,1447.50275,1813.233
6,20.0,1183.069050,399.351911,222.590,1012.69225,1144.4340,1339.47975,1935.007
7,17.0,1318.110235,414.051742,228.100,1166.18600,1317.5470,1558.01100,1994.038
8,12.0,1425.476000,524.667360,233.902,1139.70375,1515.1745,1760.44575,2030.647
9,7.0,1202.040571,508.451028,238.748,978.28600,1463.9280,1554.91200,1645.212
